# Creation de la base finale pour JS 

Votes et abstention

In [2]:
import pandas as pd
import geopandas as gpd
import os 
import matplotlib.pyplot as plt

In [4]:
dir=os.getcwd()

Densite de population

In [3]:
dens=pd.read_csv(dir+'\\brut\\metropole-densites-de-population-par-commune.csv',sep=';')
dens=dens.rename(columns={'Code INSEE':'insee',"Densité d'habitants (hab/km2)" : 'densitehabkm2'})
dens=dens[['insee','densitehabkm2']]

FileNotFoundError: [Errno 2] File /home/steve/Documents/ENSAE/Data_StoryTelling/dataviz\brut\metropole-densites-de-population-par-commune.csv does not exist: '/home/steve/Documents/ENSAE/Data_StoryTelling/dataviz\\brut\\metropole-densites-de-population-par-commune.csv'

In [ ]:
dens.insee=dens.insee.astype(int)
dens.dtypes

In [ ]:
A=dens.sort_values('insee').sort_values('insee')

In [ ]:
list(A.insee)

Niveau de vie et taux de pauvreté

In [ ]:
com=pd.read_csv(dir+'\\brut\\base-comparateur-de-territoires.csv',sep=';')
com=com.rename(columns={'CODGEO':'insee','MED16':'medianendv','TP6016':'tp'})
com=com[['insee','tp','medianendv']]

In [ ]:
com.insee=com.insee.astype(int)
com.dtypes

In [ ]:
com.head() # tp on a pas pour tout 

In [ ]:
len(com.insee.unique())

Pollution

In [ ]:
pol=pd.read_csv(dir+'\\brut\\indices_QA_commune_IDF_2017.csv',sep=',')

pol=pol.groupby('ninsee',as_index=False).mean()

pol=pol.rename(columns={'ninsee':'insee'})

In [ ]:
pol.insee=pol.insee.astype(int)

In [ ]:
pol.dtypes

Diplomes 

In [ ]:
dipl = pd.read_csv(dir+'\\brut\\base-cc-diplomes-formation-2016.csv',sep=';')

In [ ]:
dipl=dipl[['CODGEO','P16_NSCOL15P','P16_NSCOL15P_SUP']]

dipl=dipl.rename(columns={'P16_NSCOL15P':'nb_nscol','P16_NSCOL15P_SUP':'dipl_sup','CODGEO':'insee'})

dipl=dipl.assign(dipl_su_tx=dipl.dipl_sup/dipl.nb_nscol)

dipl=dipl[['insee','dipl_su_tx']]


In [ ]:
dipl.dtypes

In [ ]:
import numpy as np

In [ ]:
dipl=dipl[pd.to_numeric(dipl['insee'], errors='coerce').notnull()]

In [ ]:
dipl.insee=dipl.insee.astype(int)

Prix au mètre carré 

In [ ]:
prix=pd.read_csv(dir+'\\brut\\prixm2-communes-2017.csv',sep=',')

prix=prix[['INSEE_COM','Prixm2']]

prix=prix.rename(columns={'INSEE_COM':'insee','Prixm2':'prixm2'})

In [ ]:
prix.dtypes

In [ ]:
prix=prix[pd.to_numeric(prix['insee'], errors='coerce').notnull()]
prix.insee=prix.insee.astype(int)

In [ ]:
prix.dtypes

Impôt sur la fortune

In [ ]:
isf=pd.read_excel(dir+'\\brut\\ificom2018.xlsx',headers=True,sheet_name='définitif impot')

isf=isf[['Code commune (INSEE)','nombre de redevables','patrimoine moyen en €','impôt moyen en €']]

isf.columns=['insee','nb_redevables','patrimoineeuro','impotmoyeneuro']

In [ ]:
isf.dtypes

In [ ]:
isf['insee']= isf['insee'].transform(lambda x : x.replace(' ','')) 

In [ ]:
isf=isf[pd.to_numeric(isf['insee'], errors='coerce').notnull()]
isf.insee=isf.insee.astype(int)

In [ ]:
isf.dtypes

In [ ]:
#Base Sarah 
eco=pd.read_csv(dir+'\\brut\\data_chom_prop.csv')
eco.head()

In [ ]:
eco=eco.drop(columns=['Unnamed: 0'])
eco=eco.rename(columns={'CODGEO':'insee'})

In [ ]:
eco.dtypes

Votes

In [ ]:
#vote =pd.read_csv(dir+'\\brut\\data_final_vote.csv')
#vote=vote[['CODGEO','Tour1Candidat1']]

In [ ]:
#vote=vote.rename(columns={'CODGEO':'insee'})

In [ ]:
# Vote à la mano 
vote = pd.read_excel(dir+'\\brut\\vote_mano.xlsx')

Merge 

In [ ]:
bases=[eco,dens,dipl,pol,vote,com,isf,prix]

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')
variables= pd.DataFrame(pop.insee)
variables.insee=variables.insee.astype(int)

In [ ]:
pop.plot()

In [ ]:
for data in bases:
    variables=variables.merge(data,how='left',on='insee')

In [ ]:
variables.head()

In [ ]:
# SOucis avec les élections j'ai pas les correspondances !! 

In [ ]:
variables.drop_duplicates( inplace = True) 

In [ ]:
#variables.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
variables = variables.round(2)

In [ ]:
variables.to_csv('variables.csv')

Creation du CSV our la population slider

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')

In [ ]:
pop=pop[['insee','year','population','pct_ch','steps']]

In [ ]:
pop.to_csv('population_csv.csv')

In [ ]:
pop[pop.insee=='78561']

Professionnels de Santé

In [173]:
med=pd.read_csv(dir+'\\brut\\annuaire-et-localisation-des-professionnels-de-sante.csv',sep=';')

med=med[["Nature de l'exercice","Convention du professionnel","Coordonnées",'Adresse','Code INSEE']]

med.columns=['nature_exercice','convention','coordonnees','adresse','insee']

In [174]:
# Ici interessant de mettre des marqueurs ? Je laisse comme ça => sinon compter 
count_med=med.groupby('insee',as_index=False)['coordonnees'].count()

In [175]:
med.dtypes

nature_exercice    object
convention         object
coordonnees        object
adresse            object
insee               int64
dtype: object

In [176]:
count_med.dtypes

insee          int64
coordonnees    int64
dtype: object

In [177]:
med.insee=med.insee.astype(int)

In [178]:
med.columns

Index(['nature_exercice', 'convention', 'coordonnees', 'adresse', 'insee'], dtype='object')

In [179]:
med=med[['nature_exercice','coordonnees','insee']]

In [180]:
med=med.merge(count_med,on='insee')

In [181]:
med.columns=['nature','coordonnees','insee','nombre']

In [182]:
new= med["coordonnees"].str.split(",", n = 1, expand = True) 

In [183]:
med_data['lat'] = new[0]
med_data['lon'] = new[1]

In [184]:
med_data 

,nature,coordonnees,insee,nombre,geometry,new,lat,lon
0,Libéral activité salariée,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.970077,48.829281,2.3568
1,Libéral intégral,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.970077,48.824815,2.357629
2,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.96271,48.883061,2.329228
3,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.96271,48.833789,2.308108
4,Libéral activité salariée,48.971396,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.971396,48.853281,2.293727
...,...,...,...,...,...,...,...,...
7773,Libéral intégral,49.014456,77241,4,"POLYGON ((2.67922 49.01336, 2.68390 49.01933, ...",49.014456,48.857847,2.304228
7774,Libéral intégral,48.704353,91136,1,"POLYGON ((2.25880 48.70794, 2.25881 48.70836, ...",48.704353,48.854348,2.383557
7775,Libéral intégral,49.064301,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ...",49.064301,48.844213,2.342242
7776,Libéral intégral,49.064414,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ...",49.064414,48.865643,2.288865


In [153]:
def df_to_geojson(df, properties, lat='lat', lon='lon'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [154]:
cols = ['insee','nombre','nature']
geojson = df_to_geojson(med_data, cols)

In [160]:
import json

with open('med_data.geojson', 'a') as f:
    json.dump(geojson, f)

In [77]:
#med_data.to_csv('med_data.csv')

In [86]:
med_data["coordonnees"]= med_data["coordonnees"].str.split(",", n = 2, expand = True) 

In [85]:
med_data

,nature,coordonnees,insee,nombre,geometry
0,Libéral activité salariée,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
1,Libéral intégral,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
2,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
3,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
4,Libéral activité salariée,48.971396,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
...,...,...,...,...,...
7773,Libéral intégral,49.014456,77241,4,"POLYGON ((2.67922 49.01336, 2.68390 49.01933, ..."
7774,Libéral intégral,48.704353,91136,1,"POLYGON ((2.25880 48.70794, 2.25881 48.70836, ..."
7775,Libéral intégral,49.064301,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ..."
7776,Libéral intégral,49.064414,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ..."


In [185]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')

In [186]:
pop=pop[['insee','geometry']]

In [187]:
pop=pop.drop_duplicates()

In [188]:
pop.insee=pop.insee.astype(int)

In [192]:
med_merge=med_data[['nature','nombre','insee']]

In [193]:
med_merge = med_merge.merge(pop)

In [202]:
med_merge.columns

Index(['nature', 'nombre', 'insee', 'geometry'], dtype='object')

In [203]:
from geopandas import GeoDataFrame
med_merge = GeoDataFrame(med_merge, geometry=med_merge['geometry'])

In [207]:
med_merge.to_file("med_merge.geojson", driver='GeoJSON')

In [206]:
med_merge=med_merge[['insee','nombre','geometry']]

In [213]:
med_merge.nombre.max()

393

# Traffic (lié aux stations et non aux communes)

In [633]:
traffic=pd.read_csv(r'C:/Users/morga/Downloads/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-2e-sem.csv',sep=';')

In [647]:
pd.options.display.max_rows = 999

In [649]:
traffic[traffic['LIBELLE_ARRET']=='GARE DU NORD'].head(100)

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CATEGORIE_TITRE,NB_VALD
320,2018-11-13,100,110.0,318.0,GARE DU NORD,71410,AMETHYSTE,563.0
2018,2018-11-13,800,853.0,308.0,GARE DU NORD,71410,TST,7139.0
2357,2018-11-20,800,853.0,308.0,GARE DU NORD,71410,AUTRE TITRE,5638.0
2359,2018-11-20,800,853.0,308.0,GARE DU NORD,71410,FGT,3092.0
2360,2018-11-20,800,853.0,308.0,GARE DU NORD,71410,NAVIGO,51266.0
2632,2018-11-12,800,853.0,308.0,GARE DU NORD,71410,NAVIGO,53878.0
3668,2018-12-06,100,110.0,318.0,GARE DU NORD,71410,NAVIGO,18062.0
5330,2018-11-10,800,853.0,308.0,GARE DU NORD,71410,AUTRE TITRE,1537.0
5941,2018-12-04,100,110.0,318.0,GARE DU NORD,71410,?,15.0
5943,2018-12-04,100,110.0,318.0,GARE DU NORD,71410,NAVIGO,17286.0


In [641]:
traffic.groupby(['JOUR'],as_index=False).mean()

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,NB_VALD
0,2018-07-01,487.476232,503.792208,485.398268,503.142292
1,2018-07-02,503.555055,521.127921,484.228987,1216.861603
2,2018-07-03,504.064622,521.583591,483.714698,1323.580691
3,2018-07-04,503.149917,520.715474,482.895799,1333.968861
4,2018-07-05,504.750051,522.499897,483.653822,1341.385650
...,...,...,...,...,...
179,2018-12-27,493.062072,510.388854,482.142441,837.527874
180,2018-12-28,494.290622,511.450810,482.023077,879.479546
181,2018-12-29,487.996709,505.002060,480.192419,606.129288
182,2018-12-30,482.869348,499.573575,480.626383,473.238824


In [625]:
traffic=traffic[traffic['CODE_STIF_TRNS']==810]

CHAT = traffic[traffic['LIBELLE_ARRET'].str.startswith('CHAT')]

traffic=traffic[traffic['CODE_STIF_RES']==802]

traffic=traffic.append(CHAT)

traffic=traffic.groupby(['JOUR','LIBELLE_ARRET'],as_index=False)['NB_VALD'].sum()

traffic=traffic[['LIBELLE_ARRET','NB_VALD']]

traffic=traffic.groupby(['LIBELLE_ARRET'],as_index=False).mean()

In [626]:
liste_stations = list(traffic.LIBELLE_ARRET.unique()) + ['GARE DU NORD']

In [627]:
traffic=pd.read_csv(r'C:/Users/morga/Downloads/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-2e-sem.csv',sep=';')

In [628]:
traffic=traffic[traffic['LIBELLE_ARRET'].isin(liste_stations)]

In [629]:
traffic=traffic.groupby(['JOUR','LIBELLE_ARRET'],as_index=False)['NB_VALD'].sum()

traffic=traffic[['LIBELLE_ARRET','NB_VALD']]

traffic=traffic.groupby(['LIBELLE_ARRET'],as_index=False).mean()

In [632]:
traffic

,LIBELLE_ARRET,NB_VALD
0,AEROPORT CHARLES DE GAULLE 1,4751.359551
1,AEROPORT CHARLES DE GAULLE 2-TGV,6158.989071
2,ANTONY,11292.179348
3,ARCUEIL-CACHAN,8561.918478
4,AULNAY-SOUS-BOIS,15838.554348
5,BAGNEUX,5624.396739
6,BLANC-MESNIL,3551.989011
7,BOURG-LA-REINE,10464.875000
8,BURES-SUR-YVETTE,1350.456522
9,CHATELET-LES HALLES,54721.494565


In [585]:
perc_traffic=pd.read_csv(r'C:/Users/morga/Downloads/validations-sur-le-reseau-ferre-profils-horaires-par-jour-type-2e-sem.csv',sep=';')


In [586]:
perc_traffic=perc_traffic[perc_traffic['LIBELLE_ARRET'].isin(liste_stations)]

In [587]:
perc_traffic=perc_traffic[perc_traffic['CAT_JOUR']=='JOHV']

In [588]:
perc_traffic=perc_traffic[['LIBELLE_ARRET','pourc_validations','TRNC_HORR_60']]

In [589]:
perc_traffic=perc_traffic.groupby(['LIBELLE_ARRET','TRNC_HORR_60'],as_index=False)['pourc_validations'].mean()

In [590]:
traffic_par_heure=perc_traffic.merge(traffic,how='left')

In [591]:
traffic_par_heure=traffic_par_heure.assign(NB_VALD_HOUR=traffic_par_heure.pourc_validations*traffic_par_heure.NB_VALD)

In [592]:
traffic_par_heure.to_csv('traffic_par_heure.csv')

In [593]:
traffic_par_heure=pd.read_csv('traffic_par_heure.csv')

In [594]:
list(traffic_par_heure.LIBELLE_ARRET.unique())

['AEROPORT CHARLES DE GAULLE 1',
 'AEROPORT CHARLES DE GAULLE 2-TGV',
 'ANTONY',
 'ARCUEIL-CACHAN',
 'AULNAY-SOUS-BOIS',
 'BAGNEUX',
 'BLANC-MESNIL',
 'BOURG-LA-REINE',
 'BURES-SUR-YVETTE',
 'CHATELET-LES HALLES',
 'CHATOU-CROISSY',
 'CITE UNIVERSITAIRE',
 'COURCELLE-SUR-YVETTE',
 'DENFERT-ROCHEREAU',
 'DRANCY',
 'FONTAINE-MICHALON',
 'FONTENAY-AUX-ROSES',
 'GARE DU NORD',
 'GENTILLY',
 'GIF-SUR-YVETTE',
 'LA COURNEUVE-AUBERVILLIERS',
 'LA CROIX-DE-BERNY-FRESNES',
 'LA HACQUINIERE',
 'LA PLAINE-STADE DE FRANCE',
 'LAPLACE',
 'LE BOURGET',
 'LE GUICHET',
 'LES BACONNETS',
 'LOZERE',
 'LUXEMBOURG',
 'MASSY-PALAISEAU',
 'MASSY-VERRIERES',
 'MITRY-CLAYE',
 'ORSAY-VILLE',
 'PALAISEAU',
 'PALAISEAU-VILLEBON',
 'PARC DES EXPOSITIONS',
 'PARC-DE-SCEAUX',
 'PORT ROYAL',
 'ROBINSON',
 'SAINT-MICHEL NOTRE DAME',
 'SAINT-REMY-LES-CHEVREUSE',
 'SCEAUX',
 'SEVRAN-BEAUDOTTES',
 'SEVRAN-LIVRY',
 'VERT-GALANT',
 'VILLEPARISIS-MITRY-LE-NEUF',
 'VILLEPINTE']

In [595]:
traffic_par_heure['LIBELLE_ARRET']= traffic_par_heure['LIBELLE_ARRET'].str.replace('SAINT-MICHEL NOTRE DAME', 'SAINT-MICHEL', regex=True)

In [596]:
traffic_par_heure.to_csv('traffic_par_heure.csv')

In [597]:
import geopandas as gpd 
stations=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\stations.geojson')

In [598]:
list(stations.nom_long.unique())

['MASSY-PALAISEAU',
 'SAINT-REMY-LES-CHEVREUSE',
 'GIF-SUR-YVETTE',
 'BURES-SUR-YVETTE',
 'PORT ROYAL',
 'LA CROIX-DE-BERNY-FRESNES',
 'GENTILLY',
 'LAPLACE',
 'LA HACQUINIERE',
 'BOURG-LA-REINE',
 'PARC-DE-SCEAUX',
 'BAGNEUX',
 'MASSY-VERRIERES',
 'FONTAINE-MICHALON',
 'LES BACONNETS',
 'PALAISEAU-VILLEBON',
 'LOZERE',
 'ARCUEIL-CACHAN',
 'ANTONY',
 'ORSAY-VILLE',
 'PALAISEAU',
 'LE GUICHET',
 'COURCELLE-SUR-YVETTE',
 'DENFERT-ROCHEREAU',
 'CITE UNIVERSITAIRE',
 'SAINT-MICHEL',
 'LUXEMBOURG',
 'CHATELET-LES HALLES',
 'LA PLAINE-STADE DE FRANCE',
 'DRANCY',
 'BLANC-MESNIL',
 'PARC DES EXPOSITIONS',
 'ROBINSON',
 'SCEAUX',
 'GARE DU NORD',
 'MITRY-CLAYE',
 'VILLEPARISIS-MITRY-LE-NEUF',
 'LA COURNEUVE-AUBERVILLIERS',
 'VERT-GALANT',
 'AEROPORT CHARLES DE GAULLE 2-TGV',
 'AULNAY-SOUS-BOIS',
 'FONTENAY-AUX-ROSES',
 'AEROPORT CHARLES DE GAULLE 1',
 'LE BOURGET',
 'SEVRAN-BEAUDOTTES',
 'VILLEPINTE',
 'SEVRAN-LIVRY']

In [599]:
stations=stations.merge(traffic_par_heure,left_on='nom_long',right_on='LIBELLE_ARRET',how='right')

In [600]:
stations=stations[stations['year']==2017]

In [601]:
stations=stations[['LIBELLE_ARRET','TRNC_HORR_60','NB_VALD_HOUR','pourc_validations','geometry']]

In [602]:
stations=stations.dropna()

In [603]:
stations=stations.drop_duplicates()

In [604]:
stations

,LIBELLE_ARRET,TRNC_HORR_60,NB_VALD_HOUR,pourc_validations,geometry
8,MASSY-PALAISEAU,0H-1H,2470.910870,0.110,POINT (2.25828 48.72469)
17,MASSY-PALAISEAU,10H-11H,65479.138043,2.915,POINT (2.25828 48.72469)
26,MASSY-PALAISEAU,11H-12H,63345.169565,2.820,POINT (2.25828 48.72469)
35,MASSY-PALAISEAU,12H-13H,76148.980435,3.390,POINT (2.25828 48.72469)
44,MASSY-PALAISEAU,13H-14H,80753.859783,3.595,POINT (2.25828 48.72469)
...,...,...,...,...,...
7913,SEVRAN-LIVRY,5H-6H,49446.482568,12.010,POINT (2.53471 48.93628)
7918,SEVRAN-LIVRY,6H-7H,59039.347213,14.340,POINT (2.53471 48.93628)
7923,SEVRAN-LIVRY,7H-8H,51463.866120,12.500,POINT (2.53471 48.93628)
7928,SEVRAN-LIVRY,8H-9H,44588.293607,10.830,POINT (2.53471 48.93628)


In [605]:
new=stations.TRNC_HORR_60.str.split("H", n = 1, expand = True) 

In [606]:
stations['heure']=new[0]

In [607]:
stations=stations[['LIBELLE_ARRET','NB_VALD_HOUR','pourc_validations','heure','geometry']]

In [608]:
stations

,LIBELLE_ARRET,NB_VALD_HOUR,pourc_validations,heure,geometry
8,MASSY-PALAISEAU,2470.910870,0.110,0,POINT (2.25828 48.72469)
17,MASSY-PALAISEAU,65479.138043,2.915,10,POINT (2.25828 48.72469)
26,MASSY-PALAISEAU,63345.169565,2.820,11,POINT (2.25828 48.72469)
35,MASSY-PALAISEAU,76148.980435,3.390,12,POINT (2.25828 48.72469)
44,MASSY-PALAISEAU,80753.859783,3.595,13,POINT (2.25828 48.72469)
...,...,...,...,...,...
7913,SEVRAN-LIVRY,49446.482568,12.010,5,POINT (2.53471 48.93628)
7918,SEVRAN-LIVRY,59039.347213,14.340,6,POINT (2.53471 48.93628)
7923,SEVRAN-LIVRY,51463.866120,12.500,7,POINT (2.53471 48.93628)
7928,SEVRAN-LIVRY,44588.293607,10.830,8,POINT (2.53471 48.93628)


In [609]:
stations['heure']=stations.heure.astype(int)

C:\Users\morga\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [622]:
stations6=stations[stations['heure']==6]
stations5=stations[stations['heure']==5]
stations8=stations[stations['heure']==8]
stations10=stations[stations['heure']==10]

In [611]:
for i in list(stations.heure.unique()):
    stations_heure=stations[stations['heure']==i]
    stations_heure.to_file("stations"+str(i)+".geojson", driver='GeoJSON')

In [620]:
stations10[stations10['LIBELLE_ARRET']=='GARE DU NORD']

,LIBELLE_ARRET,NB_VALD_HOUR,pourc_validations,heure,geometry
6393,GARE DU NORD,272924.09413,3.57,10,POINT (2.35633 48.88092)


In [623]:
stations5[stations5['LIBELLE_ARRET']=='GARE DU NORD']

,LIBELLE_ARRET,NB_VALD_HOUR,pourc_validations,heure,geometry
6483,GARE DU NORD,239668.637283,3.135,5,POINT (2.35633 48.88092)


Cartes pour visualiser le résultat 

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')
pop.insee=pop.insee.astype(int)
data_map=variables.merge(pop,on='insee')

In [ ]:
data_map.columns

In [ ]:
import mapclassify
import geoplot
import matplotlib.pyplot as plt 
data_map=data_map.set_geometry('geometry')

In [ ]:
scheme = mapclassify.Quantiles(data_map.TauxChomage, k=10)
geoplot.choropleth(
    data_map, hue=data_map.TauxChomage, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux de chômage')

In [ ]:
scheme = mapclassify.Quantiles(data_map.TauxProprietaires, k=10)
geoplot.choropleth(
    data_map, hue=data_map.TauxProprietaires, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux de propriétaires')

In [ ]:
scheme = mapclassify.Quantiles(data_map.pm10, k=10)
geoplot.choropleth(
    data_map, hue=data_map.pm10, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux pollution pm10')

In [ ]:
scheme = mapclassify.Quantiles(data_map.dipl_su_tx, k=10)
geoplot.choropleth(
    data_map, hue=data_map.dipl_su_tx, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('dipl_su_tx')

In [ ]:

geoplot.choropleth(
    data_map, hue=data_map.Tour1Candidat1,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('vote_politique')